In [4]:
import carla
import math
import random
import time
import numpy as np
import open3d as o3d

ModuleNotFoundError: No module named 'open3d'

In [ ]:
#Conexion cliente-servidor
client = carla.Client('localhost', 2000) #cliente
world = client.get_world() #world reprsenta la simulacion, server

settings = world.get_settings()
print(settings)

In [ ]:
#settings de simulacion

#fixed delta - tiempo entre pasos de simulacion fijo
#modo sincrono - el server espera un tick del cliente para ejecutar un paso de simulacion

FPS = 40.0
settings.fixed_delta_seconds = 1.0/FPS
settings.synchronous_mode = True
world.apply_settings(settings)

In [ ]:
#Libreria de blueprints, para spawnear vehiculos, sensores, etc
bp_lib = world.get_blueprint_library()

In [5]:
#sensor lidar
#Velodyne HDL-64
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
#lidar_bp.set_attribute('sensor_tick', '1.0') # 1 segundo tic
lidar_bp.set_attribute('channels',str(64))
lidar_bp.set_attribute('upper_fov',str(2.0))
lidar_bp.set_attribute('lower_fov',str(-24.9))
lidar_bp.set_attribute('points_per_second',str(1300000)) #Single Return Mode
lidar_bp.set_attribute('rotation_frequency',str(10)) #frec de sim = 1/fixed_delta_time = 10 debe coincidir con la frecuencia de rotación para generar una vuelta completa por paso de simulacion
lidar_bp.set_attribute('range',str(120))
lidar_bp.set_attribute('dropoff_general_rate','0.0')
lidar_bp.set_attribute('noise_stddev', '0.01')

NameError: name 'blueprint_library' is not defined

In [ ]:
#se puede obtener la ubicacion de la camara durante la simulacion
spectator = world.get_spectator()
spectator_transform = spectator.get_transform()

In [6]:
#Spawnear el sensor
sensor_raycast = world.spawn_actor(lidar_bp, spectator_transform)

NameError: name 'world' is not defined

In [ ]:
def lidar_callback(point_cloud):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    print(point_cloud)
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]

    # Isolate the 3D data
    points = data[:, :-1]

    # We're negating the y to correclty visualize a world that matches
    # what we see in Unreal since Open3D uses a right-handed coordinate system
    points[:, :1] = -points[:, :1]
    
    mlab.points3d(points[:,0], points[:,1], points[:,2], intensity, mode='point')
    mlab.show()

In [ ]:
sensor_raycast.listen(lambda data: lidar_callback(data))

In [ ]:
world.tick()